# <span style= "color :red"> 생성적 적대 신경망 </span>
- GAN : 딥러닝의 원리를 활용해 가상 이미지를 생성하는 알고리즘<br><br>
- 진짜 같은 가짜를 만들기 위해 알고리즘 내부에서 '적대적' 경합을 진행하는 알고리즘 <br><br>
- 한쪽은 진짜 같은 가짜를 만들고, 한쪽은 진짜와 비교하면서 경합 과정을 이용하는 원리<br><br>

### <span style= "color :blue"> 가짜 제조 공장 : 생성자  </span>
- 가상의 이미지를 만들어내는 공장 <br><br>
- 초기엔 랜덤한 픽셀로 이뤄진 가짜 이미지를 만들고 판별자의 결과에 따라 지속적으로 업데이트 하며 점차 원하는 이미지를 만들어 낸다<br><br>

In [1]:
from tensorflow import keras
from keras.layers import LeakyReLU

generator = keras.Sequential()
generator.add(keras.layers.Dense(128*7*7,                   # 128 : 임의로 정한 노드의 수, 7*7 : 이미지의 최초 크기 
                                 input_dim=100,             # input_dim : 100차원 크기의 랜덤 백터
                                 activation=LeakyReLU(0.2)))# 0이하에서도 작은 값을 갖게 만드는 RELU의 보완함수  
generator.add(keras.layers.BatchNormalization())            # 데이터의 배치를 정규분포로 만드는 배치정규화 진행


generator.add(keras.layers.Reshape((7,7,128)))              # 합성곱신경망이 받아들일수 있는 형태로 바꿔주는 코드
generator.add(keras.layers.UpSampling2D())                  # 이미지의 가로세로 크기를 2배씩 늘려준다 (14,14)


generator.add(keras.layers.Conv2D(64, kernel_size = 5, padding="same"))
generator.add(keras.layers.BatchNormalization())


generator.add(keras.layers.Activation(LeakyReLU(0.2)))
generator.add(keras.layers.UpSampling2D())                  # 이미지의 가로세로 크기를 2배씩 늘려준다 (28,28)

generator.add(keras.layers.Conv2D(1, kernel_size = 5,       # 판별자로 넘길 준비를 완료 
                                  padding="same", 
                                  activation="tanh"))       # 판별자로 넘기기 전에는 tanh함수를 사용 



c:\Users\172559\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### <span style= "color :blue"> 진위를 가려내는 장치 : 판별자  </span>
- 생성자에서 넘어온 이미지가 가짜인지 진짜인지를 판별해주는 장치 
- CNN구조를 그대로 가지고와서 사용하면 됨

In [2]:
descriminator = keras.Sequential()
descriminator.add(keras.layers.Conv2D(64, kernel_size=5, strides=2, 
                                      input_shape=(28,28,1), padding="same",
                                      activation = LeakyReLU(0.2)))
descriminator.add(keras.layers.Dropout(0.3))
descriminator.add(keras.layers.Conv2D(128, kernel_size=5, strides=2, 
                                      input_shape=(28,28,1), padding="same",
                                      activation = LeakyReLU(0.2)))
descriminator.add(keras.layers.Dropout(0.3))
descriminator.add(keras.layers.Flatten())
descriminator.add(keras.layers.Dense(1, activation="sigmoid"))
descriminator.compile(loss = "binary_crossentropy", optimizer= "adam")

descriminator.trainable = False           # 판별이 끝나면 판별자 자신이 학습되지 않게끔 학습 기능을 꺼준다.

### <span style= "color :blue"> 적대적 신경망 실행하기  </span>

In [3]:
ginput = keras.layers.Input(shape=(100,))                   # 랜덤한 100개의 백터를 생성
dis_output = descriminator(generator(ginput))               # 출력되는 결관는 28*28크기의 이미지, 이후 판별자 모델의 값으로 사용
gan= keras.Model(ginput, dis_output)                        # Model함수를 이용해 입력과 출력값을 가지는 GAN모델을 생성

gan.compile(loss="binary_crossentropy", optimizer="adam")   # 참과 거짓으 구분하는 이진함수

In [21]:
cd

C:\Users\172559


In [ ]:
# 실행할수를 선언
def gan_train(epochs, batch_size, saving_interval):
    import numpy as np
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import os 
    
    os.chdir("./혼공머신딥러닝/DeepLearning/")
    
    # Mnist데이터 호출
    (x_train, _), (_ , _) = keras.datasets.mnist.load_data()
    
    # 차원을 지정 
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype("float32")
    
    # 배치정규화
    x_train = (x_train-127.5) / 127.5
    
    true = np.ones((batch_size, 1))             # 1의 레이블 값을 가지는 배열을 생성 (배치 길이만큼) 
    fake = np.zeros((batch_size, 1))   
    for i in range(epochs):
        idx = np.random.randint(0, x_train.shape[0], batch_size) # 0부터 데이터 갯수까지의 숫자 중 하나를 랜덤하게 배치사이즈만큼 반복해서 가져옴
        imgs = x_train[idx]
        
        # 판별자 모델에 판별을 시작 (입력값과 레이블을 받아서 딱 한번만 학습을 실시해 모델을 업데이트)
        d_loss_real = descriminator.train_on_batch(imgs, true)  
        
        # 가상 이미지를 판별자에 입력하는 부분
        noise = np.random.normal(0,1, (batch_size,100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = descriminator.train_on_batch(gen_imgs, fake)
        
        # 판별자와 생성자의 오차를 계산 
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)
        
        print("epoch : {}, d_loss : {},  g_loss : {}".format(i, d_loss, g_loss)) 
        
        # 중간 과정을 이미지로 저장하는 부분, 정해진 인터벌 만큼 학습되면 그때 만든 이미지를 gan_images폴더에 저장하라는 의미
        if i%saving_interval==0:
            r,c = 5,5
        noise = np.random.normal(0,1,(25,100))
        gen_imgs = 0.5*gen_imgs+.05
        
        fig, axs = plt.subplots(5,5)
        count = 0
        for j in range(5):
            for i in range(5):
                axs[j,i].imshow(gen_imgs[count,:,:,0],cmap="gray_r")
                axs[j,i].axis("off")
                count += 1
        fig.savefig("./gan_imgs/gan_mnist_{}.png".format(i))
        
# 2000번 반복 (+1하는것에 주의)
# 배치 크기는 32, 200번마다 결과가 저장
gan_train(2001,32,200)
        